In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
import seaborn as sns
from scipy.stats import pearsonr
import statistics
import copy

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut, cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.metrics import f1_score

from sklearn import svm

In [2]:
resistant = pd.read_csv('C:\\Users\\Muskan Agarwal\\Documents\\CB\\Dissertation\\Dataset\\akashM.techdatasets\\Esophageal_FU\\Esophageal_fr.txt', header=None, delimiter=r"\s+")

control = pd.read_csv('C:\\Users\\Muskan Agarwal\\Documents\\CB\\Dissertation\\Dataset\\akashM.techdatasets\\Esophageal_FU\\Esophageal_fc.txt', header=None, delimiter=r"\s+")

known = pd.read_csv("C:\\Users\\Muskan Agarwal\\Documents\\CB\\Dissertation\\Dataset\\akashM.techdatasets\\Esophageal_FU\\Esophageal_FU_known.csv")

name = pd.read_csv('C:\\Users\\Muskan Agarwal\\Documents\\CB\\Dissertation\\Dataset\\akashM.techdatasets\\Esophageal_FU\\esoph ff miRNA names.txt', header=None)


In [3]:
resistant = resistant.T
resistant

,0,1,2,3,4,5,6,7,8,9,...,837,838,839,840,841,842,843,844,845,846
0,10.4504,0.049358,6.22520,0.042483,6.62640,0.933860,7.34206,0.854209,-0.183436,2.02220,...,0.115565,0.548883,0.537978,1.250760,4.36272,7.83587,0.656886,1.502260,1.068210,0.887137
1,10.3835,0.610181,5.52450,0.293649,7.02637,1.544940,7.73616,0.174747,-0.049150,3.96454,...,0.214117,0.573786,-0.096711,0.603479,3.73999,7.47690,0.094549,1.791620,1.343690,0.878641
2,10.5907,0.033729,6.25172,0.050679,6.81719,1.691420,7.50762,0.660440,0.010409,3.78392,...,0.796520,0.541096,0.162022,0.495517,3.97091,7.72414,0.488425,1.335420,0.664434,0.241162
3,10.2806,-0.268577,6.01730,1.092900,6.90685,0.692013,7.48470,0.523381,-0.065090,2.35822,...,-0.051287,0.028108,-0.164372,0.680934,1.49676,7.39603,0.387191,1.431190,1.209130,0.523381
4,10.6200,0.070425,6.55232,0.332483,7.40726,1.046710,8.04558,0.497629,0.144394,3.20818,...,1.092280,0.823313,0.160251,1.096550,4.03725,6.87129,0.109856,1.446530,1.348870,1.485980
5,10.9251,0.173263,6.28965,0.191563,7.36378,1.268120,8.10759,0.293955,0.306186,2.38685,...,0.019173,0.903110,-0.141481,1.170470,3.36144,7.63455,0.443947,0.916832,1.303410,0.275316


In [4]:
control = control.T


In [5]:
known_control, known_resistant, index_name, index_known = [], [], [], []

for i in range(len(known)):
    for j in range(len(name)):
        if known[known.columns[1]][i] == name[0][j]:
            known_control.append(control[j])
            known_resistant.append(resistant[j])
            index_name.append(j)
            index_known.append(i)
            break


In [6]:
len(known_resistant)

20

In [7]:
columns = len(control.T)
mr = []
for i in range(columns):
    mr.append(resistant[i].mean())

mc = []
for i in range(columns):
    mc.append(control[i].mean())


In [9]:
D, F, corr, corr_known, D_known, F_known, scaled_corr, scaled_corr_known  = [], [], [], [], [], [], [], []
for x in range(columns):
    D.append(abs(mr[x] - mc[x]))
    
    if mc[x] >= mr[x]:
        F.append((mc[x] - mr[x]) / mc[x])
    else:
        F.append((mr[x] - mc[x]) / mr[x])
        
    corr.append(control[x].corr(resistant[x]))
    scaled_corr.append((1-corr[x])/2)
    
    
    
for x in range(len(known_control)):
    D_known.append(abs(known_mr[x] - known_mc[x]))
    
    if known_mc[x] >= known_mr[x]:
        F_known.append((known_mc[x] - known_mr[x]) / known_mc[x])
    else:
        F_known.append((known_mr[x] -known_mc[x]) / known_mr[x])
    
    corr_known.append(known_control[x].corr(known_resistant[x]))
    scaled_corr_known.append((1-corr_known[x])/2 )

In [10]:
P, A, B, C, M = [], [], [], [], []
for p in np.arange(0.1, 1.6, 0.1):
    for a in np.arange(0.1, 1, 0.01):
        for b in np.arange(0.1, 1, 0.01):
            for c in np.arange(0.1, 1, 0.01):
                l = []
                for x in range(len(known_control)):
                    l.append((a*D_known[x] + b*(F_known[x]**p) + c*scaled_corr_known[x]))
                P.append(p)
                A.append(a)
                B.append(b)
                C.append(c)
                M.append(sum(l) / len(l))
                
min_avg_rank = M.index(max(M))
p, a, b, c = P[min_avg_rank], A[min_avg_rank], B[min_avg_rank], C[min_avg_rank]
print(f"min_avg_rank = {min_avg_rank}, p = {p}, a= {a}, b={b}, c={c} ")

min_avg_rank = 728999, p = 0.1, a= 0.9899999999999995, b=0.9899999999999995, c=0.9899999999999995 


In [12]:
Q = []
for x in range(len(control.T)):
    Q.append((a*D[x] + b*(abs(F[x])**p) + c*scaled_corr[x]))

In [14]:
G = [(x, i) for i, x in enumerate(Q)]
G.sort(reverse = True)
E = G[:50]

In [17]:

for i in range(len(E)):
    for j in range(len(index_name)):
        if E[i][1] == index_name[j]:
            print(f'name:{name[0][index_name[j]]}, index in name:{index_name[j]}, rank: {i}')
            break
            

name:hsa-miR-192_st, index in name:253, rank: 25


In [18]:
l = int(len(F) * 0.05)
H = G[ : l]

In [20]:
mat = np.zeros((len(control) + len(resistant), len(H) + 1), dtype = 'float64')
for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
        if i < len(resistant):
            if j != mat.shape[1] - 1:
                mat[i][j] = resistant[H[j][1]][i]
            else:
                mat[i][j] = 0
        else:
            if j != mat.shape[1] - 1:
                mat[i][j] = control[H[j][1]][i - len(resistant)]
            else:
                mat[i][j] = 1

In [23]:
X, y = mat[:,:-1], mat[:,-1:]

In [24]:
X.shape, y.shape

((12, 42), (12, 1))

In [25]:
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut

# Create the Random Forest classifier
clf = RandomForestClassifier(n_estimators=2, random_state=2)

# Perform Leave-One-Out Cross-Validation
predictions = []
true_labels = []
for train_index, test_index in LeaveOneOut().split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit the classifier on the training data
    clf.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = clf.predict(X_test)
    
    # Collect the predictions and true labels
    predictions.append(y_pred[0])
    true_labels.append(y_test[0])

# Calculate accuracy, specificity, sensitivity, MCC, and F1-score
accuracy = accuracy_score(true_labels, predictions)
tn, fp, fn, tp = confusion_matrix(true_labels, predictions).ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
mcc = matthews_corrcoef(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

# Print the results
print("Accuracy:", accuracy)
print("Specificity:", specificity)
print("Sensitivity:", sensitivity)
print("MCC:", mcc)
print("F1-score:", f1)


Accuracy: 0.9166666666666666
Specificity: 1.0
Sensitivity: 0.8333333333333334
MCC: 0.8451542547285166
F1-score: 0.9090909090909091


c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change th

In [27]:
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, f1_score
from sklearn.model_selection import LeaveOneOut
import numpy as np

# Create the SVM classifier
clf = svm.SVC(kernel='linear', C=1, random_state=42)

# Perform Leave-One-Out Cross-Validation
predictions = []
true_labels = []
for train_index, test_index in LeaveOneOut().split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit the classifier on the training data
    clf.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = clf.predict(X_test)
    
    # Collect the predictions and true labels
    predictions.append(y_pred[0])
    true_labels.append(y_test[0])

# Calculate accuracy, specificity, sensitivity, MCC, and F1-score
accuracy = accuracy_score(true_labels, predictions)
tn, fp, fn, tp = confusion_matrix(true_labels, predictions).ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
mcc = matthews_corrcoef(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

# Print the results
print("Accuracy:", accuracy)
print("Specificity:", specificity)
print("Sensitivity:", sensitivity)
print("MCC:", mcc)
print("F1-score:", f1)
# print("Avg accuracy:", np.mean(scores))


Accuracy: 1.0
Specificity: 1.0
Sensitivity: 1.0
MCC: 1.0
F1-score: 1.0


c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\sklearn\ut